In [1]:
import os, re, html
import numpy as np
import pandas as pd
import folium
from folium import Popup, Tooltip, Element

# -------------------- CONFIG --------------------
CSV_PATH = "South Florida Live Local Act tax abatements map - Sheet1.csv"  # <-- change if needed

PROPERTY_COL = "Property"
COORD_COL    = "Google coordinates"
CAPTION_COL  = "Caption"

MAPBOX_STYLE = "mapbox/streets-v11"    # e.g. streets-v12, outdoors-v12, satellite-streets-v12
# ------------------------------------------------

# ---- Load Mapbox token (try %store first, then env) ----
try:
    get_ipython().run_line_magic("store", "-r map_box_api_key")
except Exception:
    pass  # not in IPython or no stored variable

if 'map_box_api_key' not in globals() or not map_box_api_key:
    map_box_api_key = (
        os.getenv("MAPBOX_TOKEN")
        or os.getenv("MAPBOX_ACCESS_TOKEN")
        or os.getenv("MAPBOX_API_KEY", "")
    )

if not map_box_api_key:
    raise ValueError("No Mapbox token found. Set %store map_box_api_key or env var MAPBOX_TOKEN / MAPBOX_ACCESS_TOKEN / MAPBOX_API_KEY.")

# ---- Load CSV ----
df = pd.read_csv(CSV_PATH)

# Strip whitespace from column names to avoid "Google coordinates " vs "Google coordinates"
df.columns = [c.strip() for c in df.columns]

missing_cols = [c for c in [PROPERTY_COL, COORD_COL, CAPTION_COL] if c not in df.columns]
if missing_cols:
    raise ValueError(
        f"Missing expected columns {missing_cols} in your CSV. Got: {df.columns.tolist()}"
    )

# Clean up property strings for display
df[PROPERTY_COL] = df[PROPERTY_COL].astype(str).str.strip()

# ---- Parse coordinates from COORD_COL into __lat / __lon ----
# Handles formats like:
#   "26.7145,-80.0523"
#   "26.7145, -80.0523"
#   "(26.7145, -80.0523)"
#   "-80.0523 26.7145"  (we'll treat the *first* number as lat, second as lon)
coord_re = re.compile(r"(-?\d+(?:\.\d+)?)\s*[, ]\s*(-?\d+(?:\.\d+)?)")

lats, lons, statuses = [], [], []

for val in df[COORD_COL]:
    if pd.isna(val):
        lats.append(np.nan)
        lons.append(np.nan)
        statuses.append("CSV_EMPTY")
        continue

    s = str(val).strip()
    m = coord_re.search(s)
    if m:
        lat = float(m.group(1))
        lon = float(m.group(2))
        lats.append(lat)
        lons.append(lon)
        statuses.append("CSV_OK")
    else:
        lats.append(np.nan)
        lons.append(np.nan)
        statuses.append("CSV_INVALID")

df["__lat"] = pd.to_numeric(lats, errors="coerce")
df["__lon"] = pd.to_numeric(lons, errors="coerce")
df["__geocode_status"] = statuses   # kept for compatibility with old logic

mapped = df.dropna(subset=["__lat", "__lon"]).copy()
print(f"Rows with coordinates: {len(mapped)} / {len(df)}")
if mapped.empty:
    raise RuntimeError("No coordinates produced from Google coordinates column — check formats in that column.")

# ---- Tooltip: show Property + Caption ----
TOOLTIP_FIELDS = [PROPERTY_COL, CAPTION_COL]

def make_tooltip_html(row) -> str:
    rows_html = []
    for label in TOOLTIP_FIELDS:
        if label not in row.index:
            continue
        val = row.get(label)
        if pd.isna(val): 
            continue
        s = str(val).strip()
        if not s or s.lower() in {"nan", "none"}:
            continue
        rows_html.append(
            f"<tr><th style='text-align:left;padding-right:6px;white-space:nowrap'>{html.escape(label)}</th>"
            f"<td>{html.escape(s)}</td></tr>"
        )
    if not rows_html:
        return "<i>No details</i>"
    return ("<div style='font:12px/1.2 Arial, sans-serif'>"
            "<table style='border-collapse:collapse'>" +
            "".join(rows_html) +
            "</table></div>")

# ---- Popup (fuller table: you can control columns here) ----
coord_cols = {"__lat", "__lon", "__geocode_status"}

# If you ONLY want Property + Caption in the popup:
DISPLAY_COLUMNS = [PROPERTY_COL, CAPTION_COL]

def _clean(v):
    if pd.isna(v): 
        return None
    s = str(v).strip()
    return s if s and s.lower() not in {"nan", "none"} else None

def make_popup_html(row):
    rows = []
    for col in DISPLAY_COLUMNS:
        val = _clean(row.get(col))
        if val is None:
            continue
        if isinstance(val, str) and val.lower().startswith(("http://","https://")):
            v = f'<a href="{val}" target="_blank" rel="noopener">{html.escape(val)}</a>'
        else:
            v = html.escape(str(val))
        rows.append(f"<tr><th style='text-align:left;padding-right:8px'>{html.escape(col)}</th><td>{v}</td></tr>")
    return "<table>" + "".join(rows) + "</table>" if rows else "<i>No details</i>"

# ---- Build map ----
center = [float(mapped["__lat"].mean()), float(mapped["__lon"].mean())]
m = folium.Map(
    location=center,
    zoom_start=8,
    control_scale=True,
    tiles=(
        f"https://api.mapbox.com/styles/v1/{MAPBOX_STYLE}/tiles/256/{{z}}/{{x}}/{{y}}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="© Mapbox © OpenStreetMap",
    name="Base"
)

title_html = '''
     <h3 align="center" style="font-size:20px; margin-top:10px; margin-bottom:5px;">
         <b>South Florida Live Local Act tax abatements</b>
     </h3>
'''
m.get_root().html.add_child(Element(title_html))

# ---- Right-hand scrollable text panel ----
side_panel_html = """
<style>
#info-panel {
    position: absolute;
    top: 70px;
    right: 10px;

    /* Natural height based on content */
    max-height: calc(100vh - 100px);  /* prevents it from exceeding screen height */
    overflow-y: auto;

    width: 360px;
    max-width: 40%;
    background-color: rgba(255, 255, 255, 0.96);
    padding: 14px 18px;
    box-shadow: 0 0 8px rgba(0,0,0,0.25);
    z-index: 9999;

    font-family: Arial, sans-serif;
    font-size: 13px;
    line-height: 1.5;
}
#info-panel h4 {
    margin-top: 0;
    margin-bottom: 8px;
    font-size: 15px;
}
#info-panel p {
    margin-top: 0;
    margin-bottom: 10px;
}
</style>


<div id="info-panel">
  <h4>About this map</h4>
  <p>Across South Florida, at least 48 multifamily properties qualified this year for a property tax abatement through the Live Local Act. Most –– if not all –– weren’t originally developed under the state’s affordable and workforce housing law. Instead, landlords sought the tax breaks after the fact.</p>
  <p>Live Local gives landlords a full property tax exemption for units rented to households earning up to 80 percent of the area median income, and a 75 percent abatement for units rented to households earning from 80 percent to 120 percent of the AMI.</p>
  <p>Some landlords dropped their asking rents in order to qualify for the Live Local tax breaks –– which also allowed them to speed up leasing amid South Florida’s market-rate apartment glut and high demand for rentals at workforce rents. Others had already planned to offer workforce-level rents, meaning they didn’t need to adjust pricing to receive the abatements.</p>
</div>
"""
m.get_root().html.add_child(Element(side_panel_html))

# ---- Add markers ----
for _, r in mapped.iterrows():
    tooltip_html = make_tooltip_html(r)

    folium.CircleMarker(
        location=(float(r["__lat"]), float(r["__lon"])),
        radius=7,
        color="red",          # circle border color
        weight=2,             # thickness of border
        fill=True,
        fill_color="red",     # fill color
        popup=Popup(make_popup_html(r), max_width=450),
        tooltip=Tooltip(tooltip_html, sticky=True, direction="top"),
    ).add_to(m)

# ---- Save ----
m.save("index.html")
print("✅ Saved map to index.html")

m  # show in notebook (if Jupyter)


Rows with coordinates: 48 / 48
✅ Saved map to index.html


In [7]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/sofla_tax_abatements_12_5_2025
